In [65]:
import json
import csv
from typing import Dict, List, Any, Optional
from datetime import datetime

def load_transport_data(json_path: str) -> List[Dict[str, Any]]:
    """승하차 정보 JSON 파일을 로드한다."""
    with open(json_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    print(f"✅ 승하차 정보 {len(data)}건 로드 완료")
    return data

def get_all_routes(data: List[Dict[str, Any]]) -> List[str]:
    """모든 노선명을 반환한다."""
    routes = list(set([item['노선명'] for item in data]))
    return sorted(routes)

def get_routes_by_type(data: List[Dict[str, Any]], route_type: str) -> List[Dict[str, Any]]:
    """구분(출근/퇴근)에 따른 노선 정보를 반환한다."""
    filtered_data = [item for item in data if route_type in item['구분']]
    return filtered_data

def get_route_details(data: List[Dict[str, Any]], route_name: str) -> List[Dict[str, Any]]:
    """특정 노선의 상세 정보를 반환한다."""
    route_data = [item for item in data if item['노선명'] == route_name]
    return sorted(route_data, key=lambda x: x['순번'])

def get_station_info(data: List[Dict[str, Any]], station_name: str) -> List[Dict[str, Any]]:
    """특정 정류장의 승하차 정보를 반환한다."""
    station_data = [item for item in data if station_name in item['정류장명']]
    return station_data

def get_passenger_count_by_route(data: List[Dict[str, Any]]) -> Dict[str, Dict[str, int]]:
    """노선별 승하차 인원 통계를 반환한다."""
    route_stats = {}
    
    for item in data:
        route_name = item['노선명']
        if route_name not in route_stats:
            route_stats[route_name] = {'승차': 0, '하차': 0, '총인원': 0}
        
        action = item['승/하차']
        count = item['인원']
        
        route_stats[route_name][action] += count
        route_stats[route_name]['총인원'] += count
    
    return route_stats

def get_peak_stations(data: List[Dict[str, Any]], limit: int = 5) -> List[Dict[str, Any]]:
    """승하차 인원이 많은 정류장 상위 N개를 반환한다."""
    station_counts = {}
    
    for item in data:
        station = item['정류장명']
        if station not in station_counts:
            station_counts[station] = 0
        station_counts[station] += item['인원']
    
    sorted_stations = sorted(station_counts.items(), key=lambda x: x[1], reverse=True)
    return [{'정류장명': station, '총인원': count} for station, count in sorted_stations[:limit]]

def get_vehicle_info(data: List[Dict[str, Any]]) -> Dict[str, List[str]]:
    """차량별 노선 정보를 반환한다."""
    vehicle_routes = {}
    
    for item in data:
        vehicle = item['차량번호']
        route = item['노선명']
        
        if vehicle not in vehicle_routes:
            vehicle_routes[vehicle] = []
        
        if route not in vehicle_routes[vehicle]:
            vehicle_routes[vehicle].append(route)
    
    return vehicle_routes

# 승하차 정보 데이터 로드
transport_data = load_transport_data("승하차정보.json")

# 사용 예시
print("=== 모든 노선 목록 ===")
all_routes = get_all_routes(transport_data)
for route in all_routes:
    print(f"- {route}")

print("\n=== 출근 노선 정보 ===")
commute_routes = get_routes_by_type(transport_data, "출근")
for route in commute_routes[:3]:  # 처음 3개만 출력
    print(f"노선: {route['노선명']}, 출발시간: {route['출발시간']}, 차량: {route['차량번호']}")

print("\n=== 노선별 승하차 통계 ===")
route_stats = get_passenger_count_by_route(transport_data)
for route, stats in route_stats.items():
    print(f"{route}: 승차 {stats['승차']}명, 하차 {stats['하차']}명, 총 {stats['총인원']}명")

print("\n=== 승하차 인원 상위 정류장 ===")
peak_stations = get_peak_stations(transport_data, 3)
for station in peak_stations:
    print(f"{station['정류장명']}: {station['총인원']}명")

✅ 승하차 정보 45건 로드 완료
=== 모든 노선 목록 ===
- 출근1호-한국대서문
- 출근2호-한국전자기술연구원
- 출근3호-판교공영주차장
- 출근4호-Meta
- 퇴근1호(전자-한양정형외과)
- 퇴근2호(세파-한국대서문)
- 퇴근3호(바이오-효성아파트)
- 퇴근4호(바이오-선경아파트1차)

=== 출근 노선 정보 ===
노선: 출근1호-한국대서문, 출발시간: 07:00, 차량: 경기12어1234
노선: 출근1호-한국대서문, 출발시간: 07:00, 차량: 경기12어1234
노선: 출근1호-한국대서문, 출발시간: 07:00, 차량: 경기12어1234

=== 노선별 승하차 통계 ===
출근1호-한국대서문: 승차 20명, 하차 20명, 총 40명
출근2호-한국전자기술연구원: 승차 20명, 하차 20명, 총 40명
출근3호-판교공영주차장: 승차 36명, 하차 36명, 총 72명
퇴근1호(전자-한양정형외과): 승차 3명, 하차 3명, 총 6명
퇴근2호(세파-한국대서문): 승차 20명, 하차 20명, 총 40명
퇴근3호(바이오-효성아파트): 승차 16명, 하차 16명, 총 32명
출근4호-Meta: 승차 18명, 하차 0명, 총 18명
퇴근4호(바이오-선경아파트1차): 승차 21명, 하차 21명, 총 42명

=== 승하차 인원 상위 정류장 ===
업스테이지 앞 정차: 136명
원평공영주차장 앞: 36명
원평공영주차장 맞은편: 33명


In [66]:
import json
import pandas as pd
from typing import Dict, List, Any

def load_commute_allowance_data(json_path: str) -> List[Dict[str, Any]]:
    """통근 수당 JSON 파일을 로드한다."""
    with open(json_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    print(f"✅ 통근 수당 정보 {len(data)}건 로드 완료")
    return data

def display_commute_allowance_summary(data: List[Dict[str, Any]]):
    """통근 수당 데이터 요약 정보를 출력한다."""
    print("=== 통근 수당 데이터 요약 ===")
    print(f"총 노선 수: {len(data)}개")
    
    # 구분별 통계
    commute_types = {}
    for item in data:
        commute_type = item['구분']
        if commute_type not in commute_types:
            commute_types[commute_type] = 0
        commute_types[commute_type] += 1
    
    print("\n구분별 노선 수:")
    for commute_type, count in commute_types.items():
        print(f"  - {commute_type}: {count}개")
    
    # 운행단가 통계
    unit_prices = [item['운행단가'] for item in data]
    print(f"\n운행단가 통계:")
    print(f"  - 최고: {max(unit_prices):,}원")
    print(f"  - 최저: {min(unit_prices):,}원")
    print(f"  - 평균: {sum(unit_prices)/len(unit_prices):,.0f}원")
    
    # 지급수당 통계
    allowances = [item['지급수당'] for item in data]
    print(f"\n지급수당 통계:")
    print(f"  - 최고: {max(allowances):,}원")
    print(f"  - 최저: {min(allowances):,}원")
    print(f"  - 평균: {sum(allowances)/len(allowances):,.0f}원")

def display_commute_allowance_table(data: List[Dict[str, Any]]):
    """통근 수당 데이터를 테이블 형태로 출력한다."""
    print("\n=== 통근 수당 상세 정보 ===")
    print(f"{'구분':<15} {'노선명':<25} {'출발시간':<10} {'운행거리':<8} {'운행단가':<10} {'지급수당':<10} {'야간수당':<10}")
    print("-" * 100)
    
    for item in data:
        print(f"{item['구분']:<15} {item['노선명']:<25} {item['출발시간']:<10} {item['운행거리']:<8} {item['운행단가']:,}원 {item['지급수당']:,}원 {item['야간수당']:,}원")

def get_commute_allowance_by_type(data: List[Dict[str, Any]], commute_type: str) -> List[Dict[str, Any]]:
    """구분(출근/퇴근)에 따른 통근 수당 정보를 반환한다."""
    filtered_data = [item for item in data if commute_type in item['구분']]
    return filtered_data

def get_route_allowance_details(data: List[Dict[str, Any]], route_name: str) -> Dict[str, Any]:
    """특정 노선의 통근 수당 상세 정보를 반환한다."""
    route_data = [item for item in data if item['노선명'] == route_name]
    if route_data:
        return route_data[0]
    return None

def calculate_total_allowance_cost(data: List[Dict[str, Any]]) -> Dict[str, int]:
    """전체 통근 수당 비용을 계산한다."""
    total_unit_price = sum(item['운행단가'] for item in data)
    total_allowance = sum(item['지급수당'] for item in data)
    total_night_allowance = sum(item['야간수당'] for item in data)
    
    return {
        '총운행단가': total_unit_price,
        '총지급수당': total_allowance,
        '총야간수당': total_night_allowance,
        '총비용': total_unit_price + total_allowance + total_night_allowance
    }

# 통근 수당 데이터 로드
commute_allowance_data = load_commute_allowance_data("통근수당.json")

# 데이터 요약 출력
display_commute_allowance_summary(commute_allowance_data)

# 상세 테이블 출력
display_commute_allowance_table(commute_allowance_data)

# 비용 계산 및 출력
print("\n=== 통근 수당 비용 분석 ===")
cost_analysis = calculate_total_allowance_cost(commute_allowance_data)
for key, value in cost_analysis.items():
    print(f"{key}: {value:,}원")

# 구분별 분석
print("\n=== 출근 노선 통근 수당 ===")
commute_routes = get_commute_allowance_by_type(commute_allowance_data, "출근")
for route in commute_routes:
    print(f"노선: {route['노선명']}")
    print(f"  출발시간: {route['출발시간']}")
    print(f"  운행단가: {route['운행단가']:,}원")
    print(f"  지급수당: {route['지급수당']:,}원")
    print(f"  야간수당: {route['야간수당']:,}원")
    print()

print("=== 퇴근 노선 통근 수당 ===")
return_routes = get_commute_allowance_by_type(commute_allowance_data, "퇴근")
for route in return_routes:
    print(f"노선: {route['노선명']}")
    print(f"  출발시간: {route['출발시간']}")
    print(f"  운행단가: {route['운행단가']:,}원")
    print(f"  지급수당: {route['지급수당']:,}원")
    print(f"  야간수당: {route['야간수당']:,}원")
    print()

✅ 통근 수당 정보 8건 로드 완료
=== 통근 수당 데이터 요약 ===
총 노선 수: 8개

구분별 노선 수:
  - 업스테이지 출근: 4개
  - 업스테이지 퇴근: 4개

운행단가 통계:
  - 최고: 88,000원
  - 최저: 55,000원
  - 평균: 73,750원

지급수당 통계:
  - 최고: 10,000원
  - 최저: 10,000원
  - 평균: 10,000원

=== 통근 수당 상세 정보 ===
구분              노선명                       출발시간       운행거리     운행단가       지급수당       야간수당      
----------------------------------------------------------------------------------------------------
업스테이지 출근        출근1호-한국대서문                07:00:00   10KM     73,000원 10,000원 15,000원
업스테이지 출근        출근2호-한국전자기술연구원            07:20:00   10KM     68,000원 10,000원 15,000원
업스테이지 출근        출근3호-판교공영주차장              07:15:00   10KM     87,000원 10,000원 15,000원
업스테이지 출근        출근4호-Meta                 06:55:00   10KM     63,000원 10,000원 15,000원
업스테이지 퇴근        퇴근1호(전자-한양정형외과)           17:15:00   10KM     87,000원 10,000원 15,000원
업스테이지 퇴근        퇴근2호(세파-한국대서문)            17:15:00   10KM     69,000원 10,000원 15,000원
업스테이지 퇴근        퇴근3호(바이오-효성아파트)           17:15:00   1

In [67]:
from openai import OpenAI
import os

client = OpenAI(
    api_key=os.getenv("UPSTAGE_API_KEY"),
    base_url="https://api.upstage.ai/v1"
)
 
stream = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "system",
            "content": """너는 버스 노선별 승하차 정보를 분석하는 전문가다. 주어진 데이터를 바탕으로 노선별 승객 수, 정류장별 이용 현황, 시간대별 패턴 등을 분석하고 인사이트를 제공한다.

            데이터: {data}

            질문: {question}

            답변:"""
        },
        {
            "role": "user",
            "content": "안녕 너는 누구니"
        }
    ],
    stream=True,
)
 
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

저는 버스 노선별 승하차 정보를 분석하는 AI 전문가입니다. 다음과 같은 업무를 수행할 수 있습니다:

1. **노선별 승객 수 분석**  
   - 특정 노선의 전체 승하차 인원 통계 제공
   - 노선별 비교 분석 (이용객 수, 혼잡도 등)

2. **정류장별 이용 현황 분석**  
   - 특정 정류장의 시간대별 승하차 패턴
   - 주요 환승 지점 또는 핫스팟 식별

3. **시간대별 패턴 분석**  
   - 출퇴근 시간/심야 시간대 등 시간대별 수요 변화
   - 주말 vs 평일 이용 추이 비교

4. **데이터 기반 인사이트 제공**  
   - 노선 증편/축소 필요성 제안
   - 신규 정류장 설치에 대한 타당성 검토
   - 혼잡 완화 방안 제시 (예: 급행 노선 도입)

예를 들어, "2023년 3월 A노선의 오전 7~9시 승하차 데이터"를 요청하시면,  
- 해당 시간대의 승차/하차 그래프  
- 주요 정류장 Top 5  
- 혼잡도 지수 (분당 탑승 인원)  
등을 상세히 분석해 드릴 수 있습니다.

데이터를 입력해주시면 즉시 분석을 시작하겠습니다! 🚍📊  
(※ 현재 실제 데이터는 없으며, 분석 방법론만 설명 가능합니다)

In [82]:
import os
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# State 정의
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    transport_data: str
    commute_allowance_data: str
    analysis_result: str
    chart_type: str

# LLM 초기화
llm = ChatOpenAI(
    model="solar-pro2",
    api_key=os.getenv("UPSTAGE_API_KEY"),
    base_url="https://api.upstage.ai/v1"
)

# 노드 함수들
def chart_type_selector(state: GraphState):
    """차트 타입 선택 노드"""
    user_question = ""
    if state["messages"]:
        user_question = state["messages"][-1].content
    
    system_prompt = f"""
    너는 교통 데이터와 통근 수당 데이터를 분석하는 전문가다. 
    
    사용자 질문: {user_question}
    
    알맞은 차트 타입을 선정해줘. 추가적인 내용 포함하지 말고, 차트 타입만 영어로 출력해줘. 
    선택 가능 차트: line_chart, bar_chart, text_summary, table
    """

    messages = [
        SystemMessage(content=system_prompt),
    ]
    
    response = llm.invoke(messages)
    
    return {
        "messages": state["messages"] + [response],
        "chart_type": response.content.strip(),
        "transport_data": state.get("transport_data", ""),
        "commute_allowance_data": state.get("commute_allowance_data", ""),
        "analysis_result": ""
    }

def generate_analytic(state: GraphState):
    """인사이트 생성 노드"""
    user_question = ""
    if state["messages"]:
        user_question = state["messages"][-2].content  # 사용자 질문은 두 번째 마지막 메시지
    
    system_prompt = f"""
    너는 교통 데이터와 통근 수당 데이터를 분석하는 전문가야. python 코드로 작성하지말고 오직 분석만 작성해줘

    차트 타입 선택 결과: {state.get('chart_type', '')}  
    line chart 일경우 데이터 형식:
    
    example:
    labels: ['January', 'February', 'March', 'April', 'May', 'June', 'July'],
    datasets: [{{
        label: 'My First Dataset',
        data: [65, 59, 80, 81, 56, 55, 40],
        fill: false,
        borderColor: 'rgb(75, 192, 192)',
        tension: 0.1
    }}]
    
    교통 데이터: {state.get('transport_data', '')}
    통근 수당 데이터: {state.get('commute_allowance_data', '')}

    사용자 질문: {user_question}
    선택된 차트 타입: {state.get('chart_type', '')}
    
    위 데이터를 바탕으로 사용자 질문에 대한 분석 결과를 제공해줘.
    아래의 output format에 맞춰 선택한 엣지 정보를 JSON으로 출력해줘, 이외에 절대 다른 내용은 출력하지 말아줘.

        output format:
        ```json
        {{
            "chart_data": {{
                "labels": ["January", "February", "March", "April", "May", "June", "July"],
                "datasets": [{{
                    "label": "My First Dataset",
                    "data": [65, 59, 80, 81, 56, 55, 40],
                    "fill": false,
                    "borderColor": "rgb(75, 192, 192)",
                    "tension": 0.1
                }}]
            }},
            "reason": "분석결과"
        }}
        ```
    """
    
    messages = [
        SystemMessage(content=system_prompt),
    ]
    
    response = llm.invoke(messages)
    
    return {
        "messages": state["messages"] + [response],
        "analysis_result": response.content,
        "chart_type": state.get("chart_type", ""),
        "transport_data": state.get("transport_data", ""),
        "commute_allowance_data": state.get("commute_allowance_data", "")
    }

# 그래프 구성
workflow = StateGraph(GraphState)

# 노드 추가
workflow.add_node("chart_type_selector", chart_type_selector)
workflow.add_node("generate_analytic", generate_analytic)

# 엣지 설정
workflow.set_entry_point("chart_type_selector")
workflow.add_edge("chart_type_selector", "generate_analytic")
workflow.add_edge("generate_analytic", END)

# 그래프 컴파일
transport_analyzer = workflow.compile()

# 실행 예시
def run_transport_analysis(user_question: str = "", transport_data: str = "", commute_allowance_data: str = ""):
    """교통 데이터 분석 실행"""
    initial_state = {
        "messages": [HumanMessage(content=user_question)] if user_question else [],
        "transport_data": transport_data,
        "commute_allowance_data": commute_allowance_data,
        "analysis_result": "",
        "chart_type": ""
    }
    
    result = transport_analyzer.invoke(initial_state)
    return result

# 테스트 실행
# 실제 데이터를 문자열로 변환하여 전달
transport_data_str = str(transport_data)
commute_allowance_data_str = str(commute_allowance_data)

# 교통 데이터 분석 실행
analysis_result = run_transport_analysis(
    user_question="노선의 수익율을 볼수 있는 라인 그래프 생성해줘",
    transport_data=transport_data_str,
    commute_allowance_data=commute_allowance_data_str
)

# 분석 결과 출줘
for message in analysis_result["messages"]:
    if hasattr(message, 'content'):
        print(f"\n{message.content}")



노선의 수익율을 볼수 있는 라인 그래프 생성해줘

line_chart

```json
{
    "chart_data": {
        "labels": ["06:55", "07:00", "07:15", "07:20", "17:15"],
        "datasets": [{
            "label": "출근 노선 수익율 (%)",
            "data": [15.87, 13.69, 11.49, 14.71, null],
            "fill": false,
            "borderColor": "rgb(75, 192, 192)",
            "tension": 0.1
        }, {
            "label": "퇴근 노선 수익율 (%)",
            "data": [null, null, null, null, 14.29],
            "fill": false,
            "borderColor": "rgb(255, 99, 132)",
            "tension": 0.1
        }]
    },
    "reason": "수익율 계산식: (지급수당 / 운행단가) * 100\n1. 출근 노선별 수익율:\n   - 출근4호: (10,000 / 63,000)*100 = 15.87%\n   - 출근1호: (10,000 / 73,000)*100 = 13.69%\n   - 출근3호: (10,000 / 87,000)*100 = 11.49%\n   - 출근2호: (10,000 / 68,000)*100 = 14.71%\n2. 퇴근 노선별 수익율:\n   - 퇴근1호: (10,000 / 87,000)*100 = 11.49%\n   - 퇴근2호: (10,000 / 69,000)*100 = 14.49%\n   - 퇴근3호: (10,000 / 55,000)*100 = 18.18%\n   - 퇴근4호: (10,000 / 88,000)*100 = 11.36%\n

In [93]:
import os
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# State 정의
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    transport_data: str
    commute_allowance_data: str
    analysis_result: str
    chart_type: str

# LLM 초기화
llm = ChatOpenAI(
    model="solar-pro2",
    api_key=os.getenv("UPSTAGE_API_KEY"),
    base_url="https://api.upstage.ai/v1"
)

# 노드 함수들
def chart_type_selector(state: GraphState):
    """차트 타입 선택 노드"""
    user_question = ""
    if state["messages"]:
        user_question = state["messages"][-1].content
    
    system_prompt = f"""
    너는 교통 데이터와 통근 수당 데이터를 분석하는 전문가다. 
    
    사용자 질문: {user_question}
    
    알맞은 차트 타입을 선정해줘. 추가적인 내용 포함하지 말고, 차트 타입만 영어로 출력해줘. 
    선택 가능 차트: line_chart, bar_chart, text_summary, table
    """

    messages = [
        SystemMessage(content=system_prompt),
    ]
    
    response = llm.invoke(messages)
    
    return {
        "messages": state["messages"] + [response],
        "chart_type": response.content.strip(),
        "transport_data": state.get("transport_data", ""),
        "commute_allowance_data": state.get("commute_allowance_data", ""),
        "analysis_result": ""
    }

def generate_analytic(state: GraphState):
    """인사이트 생성 노드"""
    user_question = ""
    if state["messages"]:
        user_question = state["messages"][-2].content  # 사용자 질문은 두 번째 마지막 메시지
    
    system_prompt = f"""
    너는 교통 데이터와 통근 수당 데이터를 분석하는 전문가야. python 코드로 작성하지말고 오직 분석만 작성해줘

    차트 타입 선택 결과: {state.get('chart_type', '')}  
    line chart 일경우 데이터 형식:
    
    교통 데이터: {state.get('transport_data', '')}
    통근 수당 데이터: {state.get('commute_allowance_data', '')}

    사용자 질문: {user_question}
    선택된 차트 타입: {state.get('chart_type', '')}
    
    
    위 데이터를 바탕으로 사용자 질문에 대한 분석 결과를 제공해줘.
    반드시 아래의 output format에 맞춰 JSON 형식으로만 출력한다. 다른 설명, 주석, 텍스트는 절대 포함하지 않는다. 오직 JSON 데이터만 출력한다.

    output format:
        {{
            "chart_data": {{
                "labels": ["January", "February", "March", "April", "May", "June", "July"],
                "datasets": [{{
                    "label": "My First Dataset",
                    "data": [65, 59, 80, 81, 56, 55, 40],
                    "fill": false,
                    "borderColor": "rgb(75, 192, 192)",
                    "tension": 0.1
                }}]
            }},
            "reason": "분석결과"
        }}
    """
    
    messages = [
        SystemMessage(content=system_prompt),
    ]
    
    response = llm.invoke(messages)
    
    return {
        "messages": state["messages"] + [response],
        "analysis_result": response.content,
        "chart_type": state.get("chart_type", ""),
        "transport_data": state.get("transport_data", ""),
        "commute_allowance_data": state.get("commute_allowance_data", "")
    }

# 그래프 구성
workflow = StateGraph(GraphState)

# 노드 추가
workflow.add_node("chart_type_selector", chart_type_selector)
workflow.add_node("generate_analytic", generate_analytic)

# 엣지 설정
workflow.set_entry_point("chart_type_selector")
workflow.add_edge("chart_type_selector", "generate_analytic")
workflow.add_edge("generate_analytic", END)

# 그래프 컴파일
transport_analyzer = workflow.compile()

# 실행 예시
def run_transport_analysis(user_question: str = "", transport_data: str = "", commute_allowance_data: str = ""):
    """교통 데이터 분석 실행"""
    initial_state = {
        "messages": [HumanMessage(content=user_question)] if user_question else [],
        "transport_data": transport_data,
        "commute_allowance_data": commute_allowance_data,
        "analysis_result": "",
        "chart_type": ""
    }
    
    result = transport_analyzer.invoke(initial_state)
    return result

# 테스트 실행
# 실제 데이터를 문자열로 변환하여 전달
transport_data_str = str(transport_data)
commute_allowance_data_str = str(commute_allowance_data)

# 교통 데이터 분석 실행
analysis_result = run_transport_analysis(
    user_question="버스 노선 최대 수익율을 볼수 있는 라인 그래프 생성해줘",
    transport_data=transport_data_str,
    commute_allowance_data=commute_allowance_data_str
)

# # 분석 결과 출줘
# for message in analysis_result["messages"]:
#     if hasattr(message, 'content'):
#         print(f"\n{message.content}")


# 분석 결과 출력
for message in analysis_result["messages"]:
    if hasattr(message, 'content'):
        print(message.content)

버스 노선 최대 수익율을 볼수 있는 라인 그래프 생성해줘
line_chart
{
    "chart_data": {
        "labels": ["출근1호", "출근2호", "출근3호", "출근4호", "퇴근1호", "퇴근2호", "퇴근3호", "퇴근4호"],
        "datasets": [{
            "label": "수익율(%)",
            "data": [14.93, 14.71, 11.49, 15.87, 11.49, 14.49, 18.18, 11.36],
            "fill": false,
            "borderColor": "rgb(75, 192, 192)",
            "tension": 0.1
        }]
    },
    "reason": "통근 수당 데이터에서 각 노선의 '지급수당'을 '운행단가'로 나눈 수익율을 계산하여 라인으로 표현. 출근 노선은 출발시간 순으로, 퇴근 노선은 노선 번호 순으로 정렬하여 표시. 계산식: (지급수당 / 운행단가) × 100"
}
